<a href="https://www.kaggle.com/code/muradpitafi/car-price-prediction-using-h2o?scriptVersionId=174392182" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/autoam-car-price-prediction/sample_submission.csv
/kaggle/input/autoam-car-price-prediction/train.csv
/kaggle/input/autoam-car-price-prediction/test.csv


In [2]:
train = pd.read_csv("/kaggle/input/autoam-car-price-prediction/train.csv")
train.head()

,model,year,motor_type,running,wheel,color,type,status,motor_volume,price
0,toyota,2022,petrol,3000 km,left,skyblue,sedan,excellent,2.0,24500
1,mercedes-benz,2014,petrol,132000 km,left,black,sedan,excellent,2.0,25500
2,kia,2018,petrol,95000 miles,left,other,sedan,excellent,2.0,11700
3,mercedes-benz,2002,petrol,137000 miles,left,golden,sedan,excellent,3.2,12000
4,mercedes-benz,2017,petrol,130000 km,left,black,sedan,good,2.0,26000


In [3]:
!pip install h2o

In [4]:
import h2o
from h2o.automl import H2OAutoML
from sklearn.metrics import mean_absolute_error

In [5]:
h2o.init()


Checking whether there is an H2O instance running at http://localhost:54321..... not found.
Attempting to start a local H2O server...
  Java Version: openjdk version "11.0.22" 2024-01-16; OpenJDK Runtime Environment (build 11.0.22+7-post-Ubuntu-0ubuntu220.04.1); OpenJDK 64-Bit Server VM (build 11.0.22+7-post-Ubuntu-0ubuntu220.04.1, mixed mode, sharing)
  Starting server from /opt/conda/lib/python3.10/site-packages/h2o/backend/bin/h2o.jar
  Ice root: /tmp/tmp1ku8pa68
  JVM stdout: /tmp/tmp1ku8pa68/h2o_unknownUser_started_from_python.out
  JVM stderr: /tmp/tmp1ku8pa68/h2o_unknownUser_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O_cluster_uptime:,03 secs
H2O_cluster_timezone:,Etc/UTC
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.46.0.1
H2O_cluster_version_age:,1 month and 14 days
H2O_cluster_name:,H2O_from_python_unknownUser_om33y6
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,7.500 Gb
H2O_cluster_total_cores:,4
H2O_cluster_allowed_cores:,4
H2O_cluster_status:,"locked, healthy"


In [6]:
train_data = h2o.import_file('/kaggle/input/autoam-car-price-prediction/train.csv')
test_data = h2o.import_file('/kaggle/input/autoam-car-price-prediction/test.csv')


Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%


In [7]:
target_column = 'price'
feature_columns = list(train_data.columns)
feature_columns.remove(target_column)


In [8]:
categorical_columns = ['model', 'motor_type', 'running', 'wheel', 'color', 'type', 'status']
for col in categorical_columns:
    train_data[col] = train_data[col].asfactor()
    test_data[col] = test_data[col].asfactor()


In [9]:
automl = H2OAutoML(max_models=20, seed=42, sort_metric='mae')
automl.train(x=feature_columns, y=target_column, training_frame=train_data)


AutoML progress: |
05:11:01.210: _train param, Dropping bad and constant columns: [wheel]

█
05:11:06.336: _train param, Dropping bad and constant columns: [wheel]

█
05:11:08.62: _train param, Dropping bad and constant columns: [wheel]

██
05:11:14.915: _train param, Dropping bad and constant columns: [wheel]

█
05:11:17.696: _train param, Dropping bad and constant columns: [wheel]

██
05:11:24.742: _train param, Dropping bad and constant columns: [wheel]

█
05:11:26.986: _train param, Dropping bad and constant columns: [wheel]

█
05:11:29.542: _train param, Dropping bad and constant columns: [wheel]

█
05:11:32.267: _train param, Dropping bad and constant columns: [wheel]

█
05:11:34.552: _train param, Dropping bad and constant columns: [wheel]

██
05:11:41.10: _train param, Dropping bad and constant columns: [wheel]

█
05:11:42.688: _train param, Dropping bad and constant columns: [wheel]

██████████████████████████████████████████
05:15:44.574: _train param, Dropping unused columns

key,value
Stacking strategy,cross_validation
Number of base models (used / total),4/6
# GBM base models (used / total),1/1
# XGBoost base models (used / total),1/1
# DeepLearning base models (used / total),1/1
# DRF base models (used / total),1/2
# GLM base models (used / total),0/1
Metalearner algorithm,GLM
Metalearner fold assignment scheme,Random
Metalearner nfolds,5


In [10]:
best_model = automl.leader

In [11]:
predictions = best_model.predict(test_data)

stackedensemble prediction progress: |███████████████████████████████████████████| (done) 100%


/opt/conda/lib/python3.10/site-packages/h2o/job.py:81: UserWarning: Test/Validation dataset column 'running' has levels not trained on: ["100500  km", "103640  miles", "105  km", "114000  miles", "118  miles", "119700  km", "12000  miles", "122000  km", "126405  miles", "1300  miles", ...67 not listed..., "6150  miles", "66800  miles", "73500  km", "77500  miles", "80500  miles", "83  km", "88700  miles", "8900  miles", "89600  miles", "94000  miles"]
  warnings.warn(w)


In [12]:
submission = pd.DataFrame({
    'Id': test_data['Id'].as_data_frame().values.flatten(),
    'price': predictions.as_data_frame().values.flatten()
})

# Save the submission file in the required format
submission.to_csv('submission.csv', index=False)

print("Submission file saved successfully.")


Submission file saved successfully.


/opt/conda/lib/python3.10/site-packages/h2o/frame.py:1983: H2ODependencyWarning: Converting H2O frame to pandas dataframe using single-thread.  For faster conversion using multi-thread, install datatable (for Python 3.9 or lower), or polars and pyarrow (for Python 3.10 or above) and activate it using:

with h2o.utils.threading.local_context(polars_enabled=True, datatable_enabled=True):
    pandas_df = h2o_df.as_data_frame()

  warnings.warn("Converting H2O frame to pandas dataframe using single-thread.  For faster conversion using"
/opt/conda/lib/python3.10/site-packages/h2o/frame.py:1983: H2ODependencyWarning: Converting H2O frame to pandas dataframe using single-thread.  For faster conversion using multi-thread, install datatable (for Python 3.9 or lower), or polars and pyarrow (for Python 3.10 or above) and activate it using:

with h2o.utils.threading.local_context(polars_enabled=True, datatable_enabled=True):
    pandas_df = h2o_df.as_data_frame()

  warnings.warn("Converting H2O f

In [13]:
h2o.shutdown()

H2O session _sid_8d05 closed.


/tmp/ipykernel_18/2900054706.py:1: H2ODeprecationWarning: Deprecated, use ``h2o.cluster().shutdown()``.
  h2o.shutdown()
